In [2]:
import pandas as pd
import numpy as np

In [98]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC, NuSVC
from sklearn.metrics import confusion_matrix,accuracy_score,roc_auc_score, balanced_accuracy_score, roc_curve

In [70]:
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [7]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.probability import FreqDist
nltk.download('punkt')
import enelvo
from enelvo.normaliser import Normaliser
import re
import unidecode

from collections import Counter
from mittens import GloVe, Mittens

[nltk_data] Downloading package punkt to /home/joao/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [43]:
df = pd.read_csv("dataset.csv")

In [44]:
df.head(20)

,sentence,category
0,"Auxílio-Doença Previdenciário, Benefícios em E...",orgão público
1,"PAGAR TODAS AS CONTAS EM ATRASO R$1.290,90.",finanças
2,Então encontraremos na próxima aula.,educação
3,Veja os resultados da categoria de ofertas do ...,indústrias
4,"Além disso, a embalagem é reutilizável e 100% ...","indústrias,varejo"
5,Na EAD Educação a distância você tem autonomia...,educação
6,Veja as condições de compra.,varejo
7,"Pensão por Morte (Art. 74/9), Benefícios em Es...",orgão público
8,A primeira agência foi aberta em 2 de janeiro ...,finanças
9,Valor das parcelas,"finanças,varejo"


# Pré-processamento

Vamos pré-processar os dados textuais, gerando um conjunto de dados mais "limpo" para alimentar o modelo. Serão removidas as strings que não são palavras, e todas as entradas serão transformadas em letras minúsculas. Além disso, a função normalise da biblioteca enelvo será utilizada para garantir que palavras potencialmente escritas em grafia não usual sejam transformadas para a ortografia padrão.

In [45]:
def pre_processamento(texto):
  
    texto_proc =  re.findall(r'\b[A-zÀ-úü]+\b', texto.lower())
    # remove stopwords
    stopwords = nltk.corpus.stopwords.words('portuguese')
    stop = set(stopwords)
    sem_stopwords = [w for w in texto_proc if w not in stop]

    # juntando os tokens novamente em formato de texto
    texto_limpo = " ".join(sem_stopwords)

    texto_proc = norm.normalise(texto_limpo)
    
    #texto_proc = unidecode.unidecode(texto_proc)
    

    return texto_proc

In [46]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')
norm = Normaliser(tokenizer='readable')

[nltk_data] Downloading package stopwords to /home/joao/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [47]:
df.sentence = [pre_processamento(sentence) for sentence in df.sentence]

In [48]:
df.head()

,sentence,category
0,auxílio doença previdenciário benefícios espéc...,orgão público
1,pagar todas contas atraso ar,finanças
2,então encontraremos próxima aula,educação
3,veja resultados categoria ofertas novo modelo ...,indústrias
4,além disso embalagem reutilizável reciclável,"indústrias,varejo"


In [49]:
class2id = {
    "fin": 0,
    "edu": 1,
    "ind": 2,
    "var": 3,
    "org": 4
}

In [50]:
id2class = {
    0: "fin",
    1: "edu",
    2: "ind",
    3: "var",
    4: "org"
}

In [51]:
total = 0
for cat in df.category:
    if "," in cat:
        total += 1

print(total)
print(len(df.category))

46
521


In [52]:
df.category.value_counts()

orgão público               140
educação                    107
indústrias                   89
varejo                       85
finanças                     54
finanças,varejo              13
educação,orgão público        9
indústrias,varejo             7
educação,indústrias           5
finanças,orgão público        4
finanças,indústrias           3
indústrias,orgão público      2
educação,finanças             2
varejo,indústrias             1
Name: category, dtype: int64

Para tratar o fato do problema ser multilabel, vamos começar empregando a estratégia mais simples: vamos selecionar apenas uma das labels quando houverem duas. A label escolhida será aquela que, dentre as duas, menos aparece no dataset, de forma a favorecer o balanceamento.

In [53]:
label_priority = {
    "org": 5,
    "edu": 4,
    "ind": 3,
    "var": 2,
    "fin": 1
}

def select_label (l1, l2):
    if label_priority[l1] < label_priority[l2]:
        return l1
    return l2


In [54]:
select_label("org", "fin")

'fin'

In [55]:
df.category = [cat.split(',') for cat in df.category]

In [56]:
df.head()

,sentence,category
0,auxílio doença previdenciário benefícios espéc...,[orgão público]
1,pagar todas contas atraso ar,[finanças]
2,então encontraremos próxima aula,[educação]
3,veja resultados categoria ofertas novo modelo ...,[indústrias]
4,além disso embalagem reutilizável reciclável,"[indústrias, varejo]"


In [57]:
new_cats = []
for cats in df.category:
    if len(cats) == 2:
        #print(cats)
        c1 = cats[0]
        c2 = cats[1]
        #print(c1[0:3],c2[0:3])
        new_cats.append([c1[0:3], c2[0:3]])
    else:
        new_cats.append([cats[0][0:3]])
        

In [58]:
final_labels = []
for cats in new_cats:
    if len(cats) == 2:
        final_labels.append(select_label(cats[0], cats[1]))
    else:
        final_labels.append(cats[0])

In [59]:
df.category = final_labels

In [60]:
df.head()

,sentence,category
0,auxílio doença previdenciário benefícios espéc...,org
1,pagar todas contas atraso ar,fin
2,então encontraremos próxima aula,edu
3,veja resultados categoria ofertas novo modelo ...,ind
4,além disso embalagem reutilizável reciclável,var


In [61]:
df.category = [class2id[cat] for cat in df.category]

In [62]:
df.head()

,sentence,category
0,auxílio doença previdenciário benefícios espéc...,4
1,pagar todas contas atraso ar,0
2,então encontraremos próxima aula,1
3,veja resultados categoria ofertas novo modelo ...,2
4,além disso embalagem reutilizável reciclável,3


In [63]:
df.category.value_counts()

4    140
1    116
2     96
3     93
0     76
Name: category, dtype: int64

# Classificação

## Bag of Words

In [73]:
data = df.drop(columns='category')

In [85]:
train, test = train_test_split(df, 
                               shuffle = True, 
                               stratify = df.category, 
                               train_size = 0.75, 
                               random_state = 50)

In [86]:
train

,sentence,category
434,conclusões decisão embargos declaração,4
263,informativos sonho virou realidade moradores b...,4
348,dois produtos adicionados sacola,3
85,história cartórios,4
36,pague primeira parcela parcelas ar nissan replay,0
...,...,...
280,projeto lei,4
361,conclusões decisão despacho,4
67,etapa educação infantil,1
159,início aulas janeiro,1


In [89]:
y_train = train.category
y_test = test.category

In [90]:
bow = CountVectorizer()
x_train = bow.fit_transform(train.sentence.values)
x_test = bow.transform(test.sentence.values)

<390x1462 sparse matrix of type '<class 'numpy.int64'>'
	with 2423 stored elements in Compressed Sparse Row format>

In [103]:
### SVC classifier
SVMC = SVC(probability=True, kernel='rbf')
#Fitting the model
SVMC.fit(x_train,y_train)

SVC(probability=True)

In [104]:
pred = SVMC.predict(x_test)
pred2 = SVMC.predict_proba(x_test)[:, 1]

acc = accuracy_score(y_test,pred)
bal_acc = balanced_accuracy_score(y_test, pred)
print(acc, bal_acc)

0.5343511450381679 0.4881807968196353


## Word2Vec

As features obtidas anteriormente utilizando bag of words não foram tão eficazes para alimentar o modelo. Vamos verificar se é possível melhorar o desempenho do classificador por meio de vetores do tipo Word2Vec. Esses vetores carregam informação contextual, e por isso é esperado que essa abordagem leve a melhores resultados. Para que cada "sentence" seja representado por um único vetor, vamos calcular o vetor médio de cada frase: tomam-se os vetores de cada palavra em uma frase, tomando o cuidado de verificar se a palavra em questão pertence ao modelo pré-treinado, e calcula-se a média entre eles.

Como modelo pré-treinado, é utilizado um Word2Vec treinado com a técnica skip gram em um corpus de língua portuguesa. O arquivo pode ser obtido em http://www.nilc.icmc.usp.br/embeddings (acesso em 29/09/2022)

In [105]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format("/home/joao/Desktop/hand_talk/sent_class/skip_s50.txt")

In [120]:
model.most_similar("projeto")

[('cronograma', 0.7738995552062988),
 ('anteprojeto', 0.7688136696815491),
 ('programa', 0.7492717504501343),
 ('plano', 0.749216616153717),
 ('substitutivo', 0.7378090620040894),
 ('amrt', 0.7197443842887878),
 ('organograma', 0.7095134854316711),
 ('tombamento', 0.7055686712265015),
 ('escopo', 0.6999985575675964),
 ('worldforge', 0.6952257752418518)]

In [138]:
def avg_vector(df):
    vectors = []
    for sentence in df.sentence.values:
        words = sentence.split(' ')
        words = [w for w in words if w in model]
        mean_vec = np.average(model[words], axis = 0)
        vectors.append(mean_vec)
    return np.array(vectors)

In [139]:
x_train = avg_vector(train)
x_test = avg_vector(test)

In [143]:
x_train.shape

(390, 50)

In [141]:
### SVC classifier
SVMC = SVC(probability=True, kernel='rbf')
#Fitting the model
SVMC.fit(x_train,y_train)

SVC(probability=True)

In [142]:
pred = SVMC.predict(x_test)
pred2 = SVMC.predict_proba(x_test)[:, 1]

acc = accuracy_score(y_test,pred)
bal_acc = balanced_accuracy_score(y_test, pred)
print(acc, bal_acc)

0.648854961832061 0.6400112349840118
